# Setup

## Function to read iMotions sensor file

In [3]:
import os
from pathlib import Path
import pandas as pd
import numpy as np

def _extract_imotions_metadata(path, metadata=None):
    """Read leading metadata lines from an iMotions CSV without loading the data."""
    metadata = metadata or []
    requested = set(metadata) if metadata else None
    meta_lines = []
    header_rows = 0
    with open(path, "r", encoding="latin1") as file:
        while True:
            line = file.readline()
            if not line:
                break
            first_cell = line.split(",", 1)[0]
            if "#" in first_cell:
                meta_lines.append(line)
                header_rows += 1
            else:
                break
    meta_dict = {}
    for raw_line in meta_lines:
        segments = raw_line.strip().split("#", 1)
        if len(segments) < 2:
            continue
        cleaned = segments[1]
        parts = cleaned.split(",")
        if len(parts) > 1:
            key = parts[0].strip()
            value = ",".join(parts[1:]).strip()
            if requested is None or key in requested:
                meta_dict[key] = value
    return meta_dict, header_rows

def read_imotions_metadata(path, metadata=None):
    """Return only the requested metadata from an iMotions CSV."""
    meta_dict, _ = _extract_imotions_metadata(path, metadata)
    return meta_dict

def read_imotions(path, metadata=None):
    """
    Reads an iMotions CSV file while extracting optional metadata fields.

    Parameters:
        path (str): Path to the iMotions CSV file.
        metadata (list[str], optional): List of metadata keys to extract.

    Returns:
        df (pd.DataFrame): The data as a DataFrame.
        meta_dict (dict): Dictionary containing requested metadata fields.
    """
    meta_dict, header_rows = _extract_imotions_metadata(path, metadata)
    df = pd.read_csv(path, header=header_rows, low_memory=True)
    return df, meta_dict

def get_files(folder, tags=['',]):
    return [f for f in os.listdir(folder) if not f.startswith('.') and all(x in f for x in tags)] 


def get_biometric_data(in_folder, results_folder):

    ######## Define ########
    # Define paths
    out_path = f"{results_folder}/"
    os.makedirs(out_path, exist_ok=True)

    respondents = [1,2,3] #define list of respondent ids

    # Define signal columns
    cols_afdex = [
                "Anger", "Contempt", "Disgust", "Fear", "Joy", "Sadness",
                "Surprise", "Engagement", "Valence", "Sentimentality",
                "Confusion", "Neutral"
        ]
    cols_eeg = ['High Engagement',
        'Low Engagement',
        'Distraction',
        'Drowsy',
        'Workload Average',
        'Frontal Asymmetry Alpha',
        ]


    #Define window lengths in seconds
    window_lengths = [3,]

    ######## Read Inputs #######
    #Get input files
    sensor_files = get_files(f'{in_folder}/Sensors/',tags=['.csv',])

    ### Begin ###

    results = []
    errors = []
    for respondent in respondents:
        error = {'respondent':respondent, 'FAC':None, 'EEG':None, 'GSR':None, 'Blinks':None, 'ET':None}
        interaction = {'respondent':respondent}
        try:
            file = [f for f in sensor_files if respondent in f][0] #may need adjustment
            df_sens_resp,_ = read_imotions(f'{in_folder}/Sensors/{file}')

            # Get sensor data per stimulus
            for task in df_sens_resp['SourceStimuliName'].unique():
                df_sens_task = df_sens_resp.loc[(df_sens_resp['SourceStimuliName']==task)]
                window = task

                # Get facial coding data
                for a in cols_afdex:
                    try:
                        interaction[f'sens_{window}_FAC_{a}_mean']=df_sens_task[a].dropna().mean()
                        auc_data = df_sens_task[['Timestamp',a]].dropna()
                        interaction[f'sens_{window}_FAC_{a}_AUC']=np.trapz(auc_data[a],x=auc_data['Timestamp'])/1000
                        interaction[f'sens_{window}_FAC_{a}_Binary']=df_sens_task[a].dropna().max()>= 50
                    except:
                        error['FAC']='Missing'

                for e in cols_eeg:
                    try:
                        interaction[f'sens_{window}_EEG_{e}_mean']=df_sens_task[e].dropna()[df_sens_int[e] > -9000].mean()
                        auc_data = df_sens_task.loc[df_sens_task[e].notna() & (df_sens_task[e] > -9000), ['Timestamp', e]]
                        interaction[f'sens_{window}_EEG_{e}_AUC']=np.trapz(auc_data[e],x=auc_data['Timestamp'])/1000
                    except:
                        error['EEG']='Missing'

                try:
                    interaction[f'sens_{window}_GSR_PeakDetected_Binary'] =1 if df_sens_task['Peak Detected'].sum()>0 else 0
                    gsr_data = df_sens_task[['Timestamp','Peak Detected']].dropna()
                    mask = gsr_data['Peak Detected'] == 1
                    segments = (mask != mask.shift()).cumsum()  # Assign unique numbers to patches
                    count_patches = gsr_data.loc[mask, 'Peak Detected'].groupby(segments).ngroup().nunique()
                    interaction[f'sens_{window}_GSR_Peaks_Count'] =count_patches
                except:
                    error['GSR']='Missing'

                try:
                    blink_data = df_sens_task[['Timestamp','Blink Detected']].dropna()
                    mask = blink_data['Blink Detected'] == 1
                    segments = (mask != mask.shift()).cumsum()  # Assign unique numbers to patches
                    count_patches = blink_data.loc[mask, 'Blink Detected'].groupby(segments).ngroup().nunique()
                    interaction[f'sens_{window}_ET_Blink_Count'] =count_patches
                    interaction[f'sens_{window}_ET_Blink_Rate'] =count_patches/((df_sens_task['Timestamp'].values[-1]-df_sens_task['Timestamp'].values[0])/(1000 * 60))
                except:
                    error['ET']='Missing'

            # TODO Get sensor data for non-interaction
            ##################################### Add this in
            results.append(interaction)
            errors.append(error)

            pass
        except IndexError:
            print(f'>>> Could not find {respondent} sensor data')
        except:
            print(f'>>> Failed {respondent}')

    results = pd.DataFrame(results)
    results.to_csv(f'{out_path}biometric_results.csv')

    errors = pd.DataFrame(errors)
    errors.to_csv(f'{out_path}errors_biometric.csv')

project_root = Path.cwd().parent
data_export_dir = project_root / "data" / "Export"

## Explanation of functions

The above functions are used to read in the sesor data files, one csv at a time, and extract single features per stimulus, and write these features to a simple results file.

The functions must be adjusted to:
- Discern between long form and short form
- Isolate key moments from timings file provided by client
- Extract time series
- Compute group-wide features such as inter-subject correlation

# Preparation
- Create naming dictionary for all stims
- Get total times of all stims
- Prepare key_moments

In [4]:
# Locate one sensor export per group for duration scanning


group_sensor_files = {}
for group_dir in sorted(data_export_dir.glob("Group *")):
    if not group_dir.is_dir():
        continue
    sensor_dirs = sorted(group_dir.glob("Analyses/*/Sensor Data"))
    csv_candidates = []
    for sensor_dir in sensor_dirs:
        csv_candidates.extend(sorted(sensor_dir.glob("*.csv")))
    group_sensor_files[group_dir.name] = csv_candidates[0] if csv_candidates else None

sensor_selection = pd.DataFrame([
    {
        "group": group,
        "sensor_file": path.name if path else None
    }
    for group, path in group_sensor_files.items()
]).sort_values("group").reset_index(drop=True)

sensor_selection

,group,sensor_file
0,Group A,001_116.csv
1,Group B,001_58.csv
2,Group C,001_114.csv
3,Group D,001_102.csv
4,Group E,001_108.csv
5,Group F,001_107.csv


In [5]:
# Collect per-group stimulus durations without aggregating across groups
duration_tables = []
issues = {}

for group, path in group_sensor_files.items():
    if path is None:
        issues[group] = "No sensor CSV found"
        continue
    try:
        df_group, _ = read_imotions(path)
    except Exception as exc:
        issues[group] = f"read_imotions failed: {exc}"
        continue

    required_cols = {"SourceStimuliName", "Timestamp"}
    if not required_cols.issubset(df_group.columns):
        issues[group] = "Missing SourceStimuliName or Timestamp"
        continue
    df_clean = df_group[["SourceStimuliName", "Timestamp"]].copy()
    df_clean = df_clean.dropna(subset=["SourceStimuliName"])
    df_clean["Timestamp"] = pd.to_numeric(df_clean["Timestamp"], errors="coerce")
    df_clean = df_clean.dropna(subset=["Timestamp"])
    if df_clean.empty:
        issues[group] = "No valid timestamp data"
        continue

    group_duration = (
        df_clean.groupby("SourceStimuliName")["Timestamp"]
        .apply(lambda s: s.max() - s.min())
        .reset_index(name="duration_ms")
    )

    if group_duration.empty:
        issues[group] = "No stimuli with duration"
        continue

    group_duration["duration_seconds"] = group_duration["duration_ms"] / 1000.0
    group_duration["duration_minutes"] = group_duration["duration_seconds"] / 60.0
    group_duration.insert(0, "group", group)
    group_duration.rename(columns={"SourceStimuliName": "stimulus_name"}, inplace=True)
    duration_tables.append(group_duration[["group", "stimulus_name", "duration_seconds", "duration_minutes"]])

if duration_tables:
    stimulus_summary = pd.concat(duration_tables, ignore_index=True)
    stimulus_summary.sort_values(["group", "stimulus_name"], inplace=True)
    stimulus_summary["duration_seconds"] = stimulus_summary["duration_seconds"].round(2)
    stimulus_summary["duration_minutes"] = stimulus_summary["duration_minutes"].round(2)
    stimulus_summary.reset_index(drop=True, inplace=True)
    stimulus_summary
else:
    print("No duration records computed.")

if issues:
    pd.DataFrame(
        {"group": list(issues.keys()), "issue": list(issues.values())}
    ).sort_values("group").reset_index(drop=True)

C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4177351198.py:55: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, low_memory=True)
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4177351198.py:55: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, low_memory=True)
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4177351198.py:55: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, low_memory=True)
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4177351198.py:55: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, header=header_rows, low_memory=True)
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4177351198.py:

In [6]:
stimulus_summary.head()

,group,stimulus_name,duration_seconds,duration_minutes
0,Group C,07 The Notebook,65.39,1.09
1,Group C,09 I Am Legend - Infected encounter,118.49,1.97
2,Group C,10 The Town - Bank robbery in nun masks,263.21,4.39
3,Group C,Abbott Elementary - S1E9 - Step Class,1291.75,21.53
4,Group C,HOME ALONE,115.09,1.92


In [7]:
issues

{'Group A': 'read_imotions failed: Unable to allocate 861. MiB for an array with shape (212, 532611) and data type float64',
 'Group B': 'read_imotions failed: Unable to allocate 838. MiB for an array with shape (212, 518407) and data type float64',
 'Group D': 'read_imotions failed: Unable to allocate 843. MiB for an array with shape (221, 499863) and data type float64',
 'Group E': 'read_imotions failed: Unable to allocate 825. MiB for an array with shape (212, 510043) and data type float64'}

In [8]:
stimulus_summary.shape

(12, 4)

In [9]:
stimulus_summary['duration_seconds'].agg(['min','max']).round(2)

min      59.82
max    1291.77
Name: duration_seconds, dtype: float64

In [10]:
stimuli_per_group = stimulus_summary.groupby('group')['stimulus_name'].nunique().reset_index(name='unique_stimuli')
stimuli_per_group

,group,unique_stimuli
0,Group C,6
1,Group F,6


In [11]:
stimulus_summary.to_csv(project_root / "results" / "stimulus_summary.csv", index=False)

# Feature Extraction

## Stimulus Annotation Overview
- `stimulus_rename` links each group-specific stimulus from `stimulus_summary` to a clean `title` and its presentation `Form` (`Long` or `Short`).
- Some titles appear in both forms; the long cut (Ã¢â€°Ë†30 min) includes the short-form key moment as an embedded segment.
- `key_moments` pinpoints, for every long-form title, when the key moment begins (`Lead-up Duration`) and how long it lasts (`Key moment Duration_LF`).
- These tables let us align short-form clips with the corresponding segment inside the long-form presentation for downstream comparisons.

## Stage 1: Demographics
We extract respondent-level identifiers and timing information from the metadata embedded in each sensor export to seed the unified view (UV). This pass scans every sensor CSV, captures study name, respondent attributes, and recording timestamps, and prepares the foundation for later feature merges.

In [12]:
import re

metadata_keys = [
    "Study name",
    "Respondent Name",
    "Respondent Group",
    "Recording time"
]

sensor_file_paths = sorted(
    (project_root / "data" / "Export").glob("Group */Analyses/*/Sensor Data/*.csv")
)

def first_segment(value):
    if value is None:
        return None
    return str(value).split(',')[0].strip()

def parse_gender(raw_gender):
    if not raw_gender:
        return None
    gender_lower = raw_gender.lower()
    if "female" in gender_lower:
        return "Female"
    if "male" in gender_lower:
        return "Male"
    if "other" in gender_lower:
        return "Other"
    return raw_gender.title()

def extract_group_letter(study_value, fallback_values):
    if study_value:
        terminal_match = re.search(r"([A-Za-z])$", study_value.strip())
        if terminal_match:
            return terminal_match.group(1).upper()
        letters = re.findall(r"[A-Za-z]", study_value)
        if letters:
            return letters[-1].upper()
    for candidate in fallback_values:
        if candidate:
            match = re.search(r"Group\s*([A-F])", str(candidate), flags=re.IGNORECASE)
            if match:
                return match.group(1).upper()
    return None

demographic_records = []

for csv_path in sensor_file_paths:
    meta = read_imotions_metadata(csv_path, metadata=metadata_keys)

    study_clean = first_segment(meta.get("Study name"))
    respondent_group_clean = first_segment(meta.get("Respondent Group"))
    group_letter = extract_group_letter(study_clean, [respondent_group_clean, csv_path.as_posix()])

    respondent_raw = first_segment(meta.get("Respondent Name"))
    respondent_value = csv_path.stem
    if respondent_raw:
        respondent_digits = re.search(r"\d+", respondent_raw)
        if respondent_digits:
            respondent_value = respondent_digits.group(0)
        else:
            respondent_value = respondent_raw

    recording_raw = meta.get("Recording time")
    date_study = None
    time_study = None
    if recording_raw:
        fragments = [frag.strip() for frag in str(recording_raw).split(',') if frag.strip()]
        date_part = None
        time_part = None
        for fragment in fragments:
            if fragment.lower().startswith("date:"):
                date_part = fragment.split(':', 1)[1].strip()
            elif fragment.lower().startswith("time:"):
                time_part = fragment.split(':', 1)[1].strip()
        if date_part and time_part:
            dt_string = f"{date_part} {time_part}"
            ts = pd.to_datetime(dt_string, utc=True, errors="coerce")
            if pd.isna(ts):
                ts = pd.to_datetime(dt_string, errors="coerce")
                if pd.notna(ts) and ts.tzinfo is None:
                    try:
                        ts = ts.tz_localize("America/Chicago")
                    except Exception:
                        ts = ts.tz_localize("UTC")
            if pd.notna(ts):
                if ts.tzinfo is None:
                    ts = ts.tz_localize("America/Chicago")
                else:
                    ts = ts.tz_convert("America/Chicago")
                date_study = ts.strftime("%m/%d/%Y")
                time_study = ts.strftime("%H:%M:%S")
            else:
                date_study = date_part
        elif date_part:
            date_study = date_part

    demographic_records.append({
        "source_file": csv_path.name,
        "group": group_letter,
        "respondent": respondent_value,
        "date_study": date_study,
        "time_study": time_study
    })

uv_stage1 = pd.DataFrame(demographic_records)

if not uv_stage1.empty:
    uv_stage1 = uv_stage1.sort_values(["group", "respondent"]).reset_index(drop=True)
    uv_stage1["respondent"] = uv_stage1["respondent"].astype(str)

uv = uv_stage1.copy()

uv_stage1

,source_file,group,respondent,date_study,time_study
0,003_104.csv,A,104,10/16/2025,18:09:03
1,002_106.csv,A,106,10/16/2025,19:35:05
2,001_116.csv,A,116,10/18/2025,12:37:40
3,006_14.csv,A,14,10/11/2025,09:32:42
4,007_3.csv,A,3,10/10/2025,09:19:22
...,...,...,...,...,...
78,005_50.csv,F,50,10/14/2025,09:54:03
79,004_60.csv,F,60,10/15/2025,09:34:06
80,003_70.csv,F,70,10/16/2025,09:49:14
81,002_85.csv,F,85,10/17/2025,14:37:41


In [13]:
# Attach supplemental demographics from grid.csv
grid_path = project_root / "data" / "grid.csv"
grid_rename_map = {
    "QB2. Age": "age",
    "QB2. Age.1": "age_group",
    "QA2. Gender": "gender",
    "QC. Ethnicity": "ethnicity",
    "QD. Income": "income_group",
    "Q1. Content Hours Per Week": "content_consumption",
    "Q2. Program Type %- Movies": "content_consumption_movies",
    "Q2. Program Type %- Series": "content_consumption_series",
    "Q2. Program Type %- Short": "content_consumption_short",
    "Comments": "grid_comments",
}

grid_raw_full = pd.read_csv(grid_path, encoding="latin1")
grid_raw_full.columns = [col.strip() for col in grid_raw_full.columns]

id_column = next((col for col in ["respondent", "Respondent", "No.", "No", "Participant", "Participant #"] if col in grid_raw_full.columns), None)
if id_column is None:
    raise ValueError("Unable to locate a respondent identifier column in grid.csv")

grid_raw_full = grid_raw_full.rename(columns={id_column: "respondent"})

available_rename_map = {orig: dest for orig, dest in grid_rename_map.items() if orig in grid_raw_full.columns}
missing_columns = sorted(set(grid_rename_map.keys()) - set(available_rename_map.keys()))
if missing_columns:
    print(f"Warning: The following columns were not found in grid.csv and will be skipped: {missing_columns}")

grid_columns = ["respondent", *available_rename_map.keys()]
grid_raw = grid_raw_full.loc[:, grid_columns].copy()

grid_raw = grid_raw.dropna(subset=["respondent"])

grid_raw["respondent"] = pd.to_numeric(grid_raw["respondent"], errors="coerce")
grid_raw = grid_raw.dropna(subset=["respondent"])
grid_raw["respondent"] = grid_raw["respondent"].astype(int).astype(str)

grid_subset = grid_raw.rename(columns=available_rename_map)

text_cols = ["age_group", "gender", "ethnicity", "income_group", "content_consumption", "grid_comments"]
for col in text_cols:
    if col in grid_subset.columns:
        grid_subset[col] = grid_subset[col].apply(lambda value: value.strip() if isinstance(value, str) else value)

numeric_cols = ["content_consumption_movies", "content_consumption_series", "content_consumption_short", "age"]
for col in numeric_cols:
    if col in grid_subset.columns:
        grid_subset[col] = pd.to_numeric(grid_subset[col], errors="coerce").astype("Int64")

grid_subset = grid_subset.drop_duplicates(subset="respondent", keep="first")

uv_stage1["respondent"] = uv_stage1["respondent"].astype(str).str.strip()
uv_stage1 = uv_stage1.merge(grid_subset, on="respondent", how="left", validate="many_to_one")
uv = uv_stage1.copy()

display_columns = [
    "respondent",
    "age",
    "gender",
    "age_group",
    "ethnicity",
    "income_group",
    "content_consumption",
    "content_consumption_movies",
    "content_consumption_series",
    "content_consumption_short",
    "grid_comments",
]
existing_display_columns = [col for col in display_columns if col in uv_stage1.columns]
uv_stage1.loc[:, existing_display_columns].head()

,respondent,age,gender,age_group,ethnicity,income_group,content_consumption,content_consumption_movies,content_consumption_series,content_consumption_short,grid_comments
0,104,59,Male,44-59,White,"$60,000 or more per year",More than 24 hours per week,10,90,0,NaN
1,106,30,Male,28-43,White,"$60,000 or more per year",3 to 12 hours per week,25,50,25,NaN
2,116,19,Male,18-27,White,"$35,000  $60,000 per year",3 to 12 hours per week,25,50,25,NaN
3,14,33,Male,28-43,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",More than 24 hours per week,20,40,40,No EEG.
4,3,34,Female,28-43,White,"$60,000 or more per year",12 to 24 hours per week,10,70,20,No EEG.


In [14]:
group_short_long_map = pd.DataFrame(
    [
        ("A", "Mad Max", "The Town"),
        ("B", "The Town", "Mad Max"),
        ("C", "The Town", "Abbot Elementary"),
        ("D", "Abbot Elementary", "The Town"),
        ("E", "Abbot Elementary", "Mad Max"),
        ("F", "Mad Max", "Abbot Elementary"),
    ],
    columns=["group", "Short Form", "Long Form"],
)
for column in ["Short Form", "Long Form"]:
    group_short_long_map[column] = group_short_long_map[column].str.strip().str.rstrip(",")

uv_stage1 = uv_stage1.merge(
    group_short_long_map,
    on="group",
    how="left",
    validate="many_to_one",
)
uv = uv_stage1.copy()

uv_stage1.loc[:, ["group", "respondent", "Short Form", "Long Form"]].head()

,group,respondent,Short Form,Long Form
0,A,104,Mad Max,The Town
1,A,106,Mad Max,The Town
2,A,116,Mad Max,The Town
3,A,14,Mad Max,The Town
4,A,3,Mad Max,The Town


In [15]:
duplicate_respondents = uv_stage1[uv_stage1.duplicated(subset="respondent", keep=False)]
if duplicate_respondents.empty:
    print("No duplicate respondents detected.")
else:
    duplicate_respondents.sort_values("respondent")

No duplicate respondents detected.


In [16]:
uv_stage1.to_csv(project_root / "results" / "uv_stage1.csv", index=False)
uv_stage1

,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,content_consumption,content_consumption_movies,content_consumption_series,content_consumption_short,grid_comments,Short Form,Long Form
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",More than 24 hours per week,10,90,0,NaN,Mad Max,The Town
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",More than 24 hours per week,20,40,40,No EEG.,Mad Max,The Town
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",12 to 24 hours per week,10,70,20,No EEG.,Mad Max,The Town
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,005_50.csv,F,50,10/14/2025,09:54:03,63,60-69,Male,Black/African American,"$60,000 or more per year",12 to 24 hours per week,70,20,10,NaN,Mad Max,Abbot Elementary
79,004_60.csv,F,60,10/15/2025,09:34:06,66,60-69,Male,White,"$35,000  $60,000 per year",3 to 12 hours per week,10,80,10,NaN,Mad Max,Abbot Elementary
80,003_70.csv,F,70,10/16/2025,09:49:14,61,60-69,Female,Black/African American,"$35,000  $60,000 per year",12 to 24 hours per week,30,30,40,NaN,Mad Max,Abbot Elementary
81,002_85.csv,F,85,10/17/2025,14:37:41,34,28-43,Female,White,"$60,000 or more per year",12 to 24 hours per week,40,60,0,NaN,Mad Max,Abbot Elementary


## Stage 2: Survey Data

### Overview
Stage 2 ingests the exposure-day survey responses while treating the Stage 1 export as the immutable base. Respondent metadata (age, gender, group, and the long/short assignment) is always reloaded from `results/uv_stage1.csv` before any survey joins. The section engineers Likert scores, familiarity composites, and open-ended extracts so downstream modeling can combine perceptual metrics with the demographic scaffold without relying on sensor features.


### Workflow Summary
1. Reload `results/uv_stage1.csv` to obtain the canonical respondent roster and long/short stimulus assignment.
2. Build the survey metadata lookup (question map, polarity, subscales) to drive scoring logic.
3. Ingest every group-level `MERGED_SURVEY_RESPONSE_MATRIX` export, harmonize respondent IDs, and apply the rename map.
4. Score Likert, familiarity, and recency responses; compute enjoyment composites; and archive open-ended text separately.
5. Merge the numeric survey features onto the Stage 1 base, logging missing or duplicate respondents to `uv_stage2_issues.csv`.
6. Write Stage 2 deliverables: `uv_stage2_features.csv`, `uv_stage2_open_ended.csv`, and the combined `uv_stage2.csv`.


In [17]:
stage1_path = project_root / "results" / "uv_stage1.csv"
if not stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 output not found at {stage1_path}.")
uv_stage1_base = pd.read_csv(stage1_path)
uv_stage1_base["respondent"] = uv_stage1_base["respondent"].astype(str).str.strip()
uv_stage1 = uv_stage1_base.copy()
uv_stage1_base.shape
uv_stage1_base.head()


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,content_consumption,content_consumption_movies,content_consumption_series,content_consumption_short,grid_comments,Short Form,Long Form
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",More than 24 hours per week,10,90,0,NaN,Mad Max,The Town
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",3 to 12 hours per week,25,50,25,NaN,Mad Max,The Town
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",More than 24 hours per week,20,40,40,No EEG.,Mad Max,The Town
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",12 to 24 hours per week,10,70,20,No EEG.,Mad Max,The Town


In [18]:
import csv
from collections import defaultdict
import re

survey_rename_map = pd.read_csv(project_root / "data" / "survey_column_rename_stage3.csv")
survey_questions = pd.read_csv(project_root / "data" / "survey_questions.csv")

survey_questions["question_code"] = survey_questions["question_code"].astype(str).str.strip()
survey_questions["question_type"] = survey_questions["question_type"].str.lower()
survey_questions["subscale"] = survey_questions["subscale"].fillna("")
survey_questions["polarity"] = survey_questions["polarity"].fillna("")

survey_metadata = (
    survey_rename_map
    .merge(survey_questions, on="question_code", how="left", suffixes=("", "_details"))
)

survey_metadata["question_type"] = survey_metadata["question_type"].fillna("likert")
survey_metadata["subscale"] = survey_metadata["subscale"].fillna("")
survey_metadata["polarity"] = survey_metadata["polarity"].fillna("")
survey_metadata_lookup = (
    survey_metadata
    .drop_duplicates(subset=["target_column"])
    .set_index("target_column")
)

survey_files = sorted(
    (project_root / "data" / "Export").glob("Group */Analyses/*/Survey/MERGED_SURVEY_RESPONSE_MATRIX-*.txt")
)

if not survey_files:
    raise FileNotFoundError("No survey response text files detected under data/Export/*/Survey/.")

len(survey_files)

12

In [19]:
LIKERT_PATTERN = re.compile(r"^\s*(\d+)(?:\.\d+)?")

LIKERT_KEYWORDS = [
    ("strongly disagree", 1.0),
    ("disagree", 2.0),
    ("neither agree nor disagree", 3.0),
    ("strongly agree", 5.0),
    ("agree", 4.0),
]

FAMILIARITY_KEY_PATTERNS = [
    (0.0, ("never heard", "not familiar")),
    (1.0, ("heard of it, but never watched", "heard of it only")),
    (2.0, ("seen a clip", "seen clips", "seen part")),
    (3.0, ("watched it in full", "just once")),
    (4.0, ("watched multiple", "very familiar")),
]

LASTWATCHED_KEY_PATTERNS = [
    (4.0, ("past week",)),
    (3.0, ("past month", "past 6 months", "past six months")),
    (2.0, ("past 3 months", "past three months")),
    (1.0, ("more than 3 months", "over 3 months")),
    (0.0, ("more than 6 months", "don't remember", "never watched this movie in full")),
]


def _clean_response(value):
    if pd.isna(value):
        return np.nan
    text = str(value).strip()
    if not text or text.upper() == "EMPTY FIELD":
        return np.nan
    return text


def _parse_likert_value(value):
    text = _clean_response(value)
    if pd.isna(text):
        return np.nan
    match = LIKERT_PATTERN.match(text)
    if match:
        return float(match.group(1))
    lowered = text.lower()
    for keyword, score in LIKERT_KEYWORDS:
        if keyword in lowered:
            return score
    try:
        return float(text)
    except ValueError:
        return np.nan


def _score_familiarity(value):
    text = _clean_response(value)
    if pd.isna(text):
        return np.nan
    match = LIKERT_PATTERN.match(text)
    if match:
        numeric = float(match.group(1))
        return float(np.clip(numeric - 1.0, 0.0, 4.0))
    lowered = text.lower()
    for score, patterns in FAMILIARITY_KEY_PATTERNS:
        if any(pattern in lowered for pattern in patterns):
            return score
    try:
        numeric = float(text)
        return float(np.clip(numeric - 1.0, 0.0, 4.0))
    except ValueError:
        return np.nan


def _score_last_watched(value):
    text = _clean_response(value)
    if pd.isna(text):
        return np.nan
    match = LIKERT_PATTERN.match(text)
    if match:
        numeric = float(match.group(1))
        return float(np.clip(numeric - 1.0, 0.0, 4.0))
    lowered = text.lower()
    for score, patterns in LASTWATCHED_KEY_PATTERNS:
        if any(pattern in lowered for pattern in patterns):
            return score
    try:
        numeric = float(text)
        return float(np.clip(numeric - 1.0, 0.0, 4.0))
    except ValueError:
        return np.nan


def _reverse_likert(value):
    if pd.isna(value):
        return np.nan
    return 6.0 - float(value)


In [20]:
def _extract_group_letter(path: Path) -> str:
    for part in path.parts:
        if part.startswith("Group ") and "-" not in part:
            return part.split()[-1].strip().upper()
    raise ValueError(f"Unable to determine group letter from path: {path}")


def _rename_survey_columns(df: pd.DataFrame, group_letter: str) -> pd.DataFrame:
    rename_subset = survey_metadata.loc[survey_metadata["group"] == group_letter]
    rename_dict = {
        raw: target
        for raw, target in zip(rename_subset["raw_column"], rename_subset["target_column"])
        if raw in df.columns
    }
    df = df.rename(columns=rename_dict)
    columns_to_keep = [
        "respondent",
        "survey_group",
        "survey_study",
        "survey_gender",
        "survey_age",
        "survey_file",
        *sorted(rename_dict.values()),
    ]
    existing_columns = [col for col in columns_to_keep if col in df.columns]
    return df.loc[:, existing_columns]


def _load_survey_file(path: Path) -> pd.DataFrame:
    # Some open-ended answers contain newline characters and stray quotes; use python engine with minimal parsing assumptions.
    return pd.read_csv(
        path,
        sep="\t",
        dtype=str,
        engine="python",
        quoting=csv.QUOTE_NONE,
        encoding="utf-8",
        on_bad_lines="warn",
    )


survey_frames = []

for survey_path in survey_files:
    group_letter = _extract_group_letter(survey_path)
    df = _load_survey_file(survey_path)
    df.columns = [col.strip() for col in df.columns]
    df = df.replace({"EMPTY FIELD": np.nan})
    df["RESPONDENT"] = df["RESPONDENT"].astype(str).str.strip()
    df = df.rename(
        columns={
            "RESPONDENT": "respondent",
            "GROUP": "survey_group",
            "STUDY": "survey_study",
            "GENDER": "survey_gender",
            "AGE": "survey_age",
        }
    )
    if "survey_group" not in df.columns:
        df["survey_group"] = group_letter
    df["survey_group"] = df["survey_group"].fillna(group_letter).astype(str).str.strip()
    if "survey_study" in df.columns:
        df["survey_study"] = df["survey_study"].astype(str).str.strip()
    else:
        df["survey_study"] = np.nan
    if "survey_gender" in df.columns:
        df["survey_gender"] = df["survey_gender"].astype(str).str.strip()
    else:
        df["survey_gender"] = np.nan
    df["survey_file"] = survey_path.name
    if "survey_age" in df.columns:
        df["survey_age"] = pd.to_numeric(df["survey_age"], errors="coerce")
    else:
        df["survey_age"] = np.nan
    df = _rename_survey_columns(df, group_letter)
    survey_frames.append(df)

survey_responses = pd.concat(survey_frames, ignore_index=True)
survey_responses

C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\1871314504.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"EMPTY FIELD": np.nan})
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\1871314504.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"EMPTY FIELD": np.nan})
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\1871314504.py:48: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.in

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E12,Long_The Town_Survey_Enjoyment_E14,Long_The Town_Survey_Enjoyment_E15,...,Short_Abbot Elementary_Survey_Enjoyment_E6,Short_Abbot Elementary_Survey_Enjoyment_E8,Short_Abbot Elementary_Survey_Enjoyment_WBD1,Short_Abbot Elementary_Survey_Enjoyment_WBD2,Short_Abbot Elementary_Survey_Enjoyment_WBD3,Short_Abbot Elementary_Survey_Enjoyment_WBD4,Short_Abbot Elementary_Survey_Enjoyment_WBD5,Short_Abbot Elementary_Survey_Familiarity_F1,Short_Abbot Elementary_Survey_Familiarity_F2,Short_Abbot Elementary_Survey_Familiarity_F3
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4 = Agree,4 = Agree,4 = Agree,4 = Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,4 = Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
survey_responses["respondent"] = survey_responses["respondent"].astype(str).str.strip()

duplicate_ids = sorted(survey_responses.loc[survey_responses["respondent"].duplicated(), "respondent"].unique())
if duplicate_ids:
    print(f"Warning: duplicate survey rows detected for respondents: {duplicate_ids}")

survey_numeric = survey_responses.drop_duplicates(subset=["respondent"], keep="first").copy()
survey_numeric

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E12,Long_The Town_Survey_Enjoyment_E14,Long_The Town_Survey_Enjoyment_E15,...,Short_Abbot Elementary_Survey_Enjoyment_E6,Short_Abbot Elementary_Survey_Enjoyment_E8,Short_Abbot Elementary_Survey_Enjoyment_WBD1,Short_Abbot Elementary_Survey_Enjoyment_WBD2,Short_Abbot Elementary_Survey_Enjoyment_WBD3,Short_Abbot Elementary_Survey_Enjoyment_WBD4,Short_Abbot Elementary_Survey_Enjoyment_WBD5,Short_Abbot Elementary_Survey_Familiarity_F1,Short_Abbot Elementary_Survey_Familiarity_F2,Short_Abbot Elementary_Survey_Familiarity_F3
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4 = Agree,4 = Agree,4 = Agree,4 = Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,4 = Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,5 = Strongly Agree,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
for column, meta in survey_metadata_lookup.iterrows():
    if column not in survey_numeric.columns:
        continue

    question_code = (meta.get("question_code") or "").strip().upper()
    question_type = (meta.get("question_type") or "").strip().lower()
    polarity = (meta.get("polarity") or "").strip().lower()

    if question_code in {"F1", "F3"} or column.endswith("_Survey_Familiarity_F1") or column.endswith("_Survey_Familiarity_F3"):
        survey_numeric[column] = survey_numeric[column].apply(_score_familiarity)
    elif question_code == "F2" or column.endswith("_Survey_Familiarity_F2"):
        survey_numeric[column] = survey_numeric[column].apply(_score_last_watched)
    elif question_type == "likert":
        survey_numeric[column] = survey_numeric[column].apply(_parse_likert_value)
        if polarity == "negative":
            survey_numeric[column] = survey_numeric[column].apply(_reverse_likert)

def _clip_zero_to_four(value):
    if pd.isna(value):
        return np.nan
    try:
        numeric = float(value)
    except (TypeError, ValueError):
        return np.nan
    return float(np.clip(numeric, 0.0, 4.0))

familiarity_columns = [
    column
    for column in survey_numeric.columns
    if column.endswith("_Survey_Familiarity_F1")
    or column.endswith("_Survey_Familiarity_F2")
    or column.endswith("_Survey_Familiarity_F3")
]

for column in familiarity_columns:
    survey_numeric[column] = survey_numeric[column].apply(_clip_zero_to_four)

survey_numeric

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E12,Long_The Town_Survey_Enjoyment_E14,Long_The Town_Survey_Enjoyment_E15,...,Short_Abbot Elementary_Survey_Enjoyment_E6,Short_Abbot Elementary_Survey_Enjoyment_E8,Short_Abbot Elementary_Survey_Enjoyment_WBD1,Short_Abbot Elementary_Survey_Enjoyment_WBD2,Short_Abbot Elementary_Survey_Enjoyment_WBD3,Short_Abbot Elementary_Survey_Enjoyment_WBD4,Short_Abbot Elementary_Survey_Enjoyment_WBD5,Short_Abbot Elementary_Survey_Familiarity_F1,Short_Abbot Elementary_Survey_Familiarity_F2,Short_Abbot Elementary_Survey_Familiarity_F3
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
subscale_columns = defaultdict(list)
overall_enjoyment_columns = defaultdict(list)

for column, meta in survey_metadata_lookup.iterrows():
    if column not in survey_numeric.columns:
        continue
    if meta.get("topic") != "enjoyment":
        continue
    if (meta.get("question_type") or "").strip().lower() != "likert":
        continue
    prefix = column.split("_Survey_")[0]
    subscale = (meta.get("subscale") or "").strip()
    overall_enjoyment_columns[prefix].append(column)
    if subscale:
        subscale_columns[(prefix, subscale)].append(column)

for (prefix, subscale), cols in subscale_columns.items():
    values = survey_numeric[cols]
    sum_col = f"{prefix}_Survey_{subscale}_Sum"
    count_col = f"{prefix}_Survey_{subscale}_Count"
    mean_col = f"{prefix}_Survey_{subscale}_Mean"
    norm_col = f"{prefix}_Survey_{subscale}_Normalized"
    count_values = values.notna().sum(axis=1)
    sum_values = values.sum(axis=1, min_count=1)
    survey_numeric[count_col] = count_values
    survey_numeric[sum_col] = sum_values
    survey_numeric[mean_col] = np.where(count_values > 0, sum_values / count_values, np.nan)
    survey_numeric[norm_col] = np.where(
        count_values > 0,
        np.clip((sum_values - count_values) / (4.0 * count_values), 0, 1),
        np.nan,
    )

for prefix, cols in overall_enjoyment_columns.items():
    values = survey_numeric[cols]
    sum_col = f"{prefix}_Survey_EnjoymentComposite_Sum"
    count_col = f"{prefix}_Survey_EnjoymentComposite_Count"
    mean_col = f"{prefix}_Survey_EnjoymentComposite_Mean"
    norm_col = f"{prefix}_Survey_EnjoymentComposite_Normalized"
    norm_corrected_col = f"{prefix}_Survey_EnjoymentComposite_NormalizedCorrected"
    corrected_col = f"{prefix}_Survey_EnjoymentComposite_Corrected"
    count_values = values.notna().sum(axis=1)

    raw_components = pd.DataFrame(index=values.index, dtype=float)
    corrected_components = pd.DataFrame(index=values.index, dtype=float)

    for column in cols:
        polarity_meta = ""
        if column in survey_metadata_lookup.index:
            polarity_meta = (survey_metadata_lookup.loc[column].get("polarity") or "").strip().lower()
        if column in survey_responses.columns:
            raw_series = survey_responses.loc[values.index, column]
            parsed_series = raw_series.apply(_parse_likert_value)
        else:
            fallback_series = pd.to_numeric(survey_numeric.loc[values.index, column], errors="coerce")
            if polarity_meta == "negative":
                parsed_series = fallback_series.apply(_reverse_likert)
            else:
                parsed_series = fallback_series
        raw_components[column] = pd.to_numeric(parsed_series, errors="coerce")
        corrected_series = pd.to_numeric(parsed_series, errors="coerce")
        if polarity_meta == "negative":
            corrected_series = corrected_series.apply(_reverse_likert)
        corrected_components[column] = corrected_series

    raw_sum_values = raw_components.sum(axis=1, min_count=1)
    corrected_sum = corrected_components.sum(axis=1, min_count=1)
    raw_normalized = np.where(
        count_values > 0,
        np.clip((raw_sum_values - count_values) / (4.0 * count_values), 0, 1),
        np.nan,
    )
    corrected_normalized = np.where(
        count_values > 0,
        np.clip((corrected_sum - count_values) / (4.0 * count_values), 0, 1),
        np.nan,
    )
    corrected_mean = np.where(count_values > 0, corrected_sum / count_values, np.nan)

    survey_numeric[count_col] = count_values
    survey_numeric[sum_col] = raw_sum_values
    survey_numeric[corrected_col] = corrected_sum
    survey_numeric[mean_col] = corrected_mean
    survey_numeric[norm_col] = raw_normalized
    survey_numeric[norm_corrected_col] = corrected_normalized

survey_numeric

C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\3342736310.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[count_col] = count_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\3342736310.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[sum_col] = sum_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\3342736310.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E12,Long_The Town_Survey_Enjoyment_E14,Long_The Town_Survey_Enjoyment_E15,...,Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected,Long_Abbot Elementary_Survey_EnjoymentComposite_Mean,Long_Abbot Elementary_Survey_EnjoymentComposite_Normalized,Long_Abbot Elementary_Survey_EnjoymentComposite_NormalizedCorrected,Short_Abbot Elementary_Survey_EnjoymentComposite_Count,Short_Abbot Elementary_Survey_EnjoymentComposite_Sum,Short_Abbot Elementary_Survey_EnjoymentComposite_Corrected,Short_Abbot Elementary_Survey_EnjoymentComposite_Mean,Short_Abbot Elementary_Survey_EnjoymentComposite_Normalized,Short_Abbot Elementary_Survey_EnjoymentComposite_NormalizedCorrected
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,4.0,4.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,5.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,43.0,3.583333,0.687500,0.645833,0,NaN,NaN,NaN,NaN,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,59.0,4.916667,0.895833,0.979167,0,NaN,NaN,NaN,NaN,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,53.0,4.416667,0.770833,0.854167,0,NaN,NaN,NaN,NaN,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,47.0,3.916667,0.645833,0.729167,0,NaN,NaN,NaN,NaN,NaN


In [24]:
familiarity_prefixes = sorted({
    column.split("_Survey_")[0]
    for column in survey_numeric.columns
    if column.endswith("_Survey_Familiarity_F1")
})

for prefix in familiarity_prefixes:
    f1_col = f"{prefix}_Survey_Familiarity_F1"
    f2_col = f"{prefix}_Survey_Familiarity_F2"
    if f1_col not in survey_numeric.columns or f2_col not in survey_numeric.columns:
        continue
    c1_col = f"{prefix}_Survey_Familiarity_C1"
    count_col = f"{prefix}_Survey_Familiarity_C1_Count"
    norm_col = f"{prefix}_Survey_Familiarity_C1_Normalized"
    pair = survey_numeric[[f1_col, f2_col]]
    sum_values = pair.fillna(0).sum(axis=1)
    count_values = pair.notna().sum(axis=1)
    survey_numeric[c1_col] = sum_values
    survey_numeric[count_col] = count_values
    survey_numeric[norm_col] = np.where(
        count_values > 0,
        np.clip(sum_values / (4 * count_values), 0, 1),
        np.nan,
    )

survey_numeric

C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4132188637.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[c1_col] = sum_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4132188637.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  survey_numeric[count_col] = count_values
C:\Users\ashra\AppData\Local\Temp\ipykernel_33984\4132188637.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joini

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E12,Long_The Town_Survey_Enjoyment_E14,Long_The Town_Survey_Enjoyment_E15,...,Long_The Town_Survey_Familiarity_C1_Normalized,Short_Abbot Elementary_Survey_Familiarity_C1,Short_Abbot Elementary_Survey_Familiarity_C1_Count,Short_Abbot Elementary_Survey_Familiarity_C1_Normalized,Short_Mad Max_Survey_Familiarity_C1,Short_Mad Max_Survey_Familiarity_C1_Count,Short_Mad Max_Survey_Familiarity_C1_Normalized,Short_The Town_Survey_Familiarity_C1,Short_The Town_Survey_Familiarity_C1_Count,Short_The Town_Survey_Familiarity_C1_Normalized
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,0.000,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,0.750,0.0,0,NaN,4.0,2,0.500,0.0,0,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,4.0,4.0,...,0.000,0.0,0,NaN,1.0,2,0.125,0.0,0,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,5.0,5.0,...,0.000,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,0.125,0.0,0,NaN,3.0,2,0.375,0.0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,5.0,2,0.625,0.0,0,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,3.0,2,0.375,0.0,0,NaN
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,0.0,0,NaN,0.0,2,0.000,0.0,0,NaN


In [25]:
open_ended_columns = [
    column
    for column, meta in survey_metadata_lookup.iterrows()
    if column in survey_numeric.columns and (meta.get("question_type") or "").strip().lower() == "open ended"
]

survey_open_ended = survey_responses[[
    "respondent",
    "survey_group",
    "survey_study",
    "survey_gender",
    "survey_age",
    "survey_file",
    *open_ended_columns,
]].copy()

survey_features = survey_numeric.drop(columns=[col for col in open_ended_columns if col in survey_numeric.columns])

# Integrate screening familiarity composites
screening_path = project_root / "results" / "individual_composite_scores.csv"
if screening_path.exists():
    screening_raw = pd.read_csv(screening_path)
    screening_raw["respondent"] = screening_raw["respondent"].astype(str).str.strip()

    screening_value_columns = [
        col
        for col in screening_raw.columns
        if col.endswith("_Survey_Familiarity_F1")
        or col.endswith("_Survey_Familiarity_F2")
        or col.endswith("_Survey_Familiarity_F3")
        or col.endswith("_Survey_Familiarity_C1")
    ]

    respondent_groups = (
        uv_stage1
        .loc[:, ["respondent", "group"]]
        .assign(group=lambda df: df["group"].astype(str).str.strip().str.upper())
        .set_index("respondent")
        .to_dict()
    )["group"]

    title_normalization = {
        "mad max fury road": "Mad Max",
        "mad max": "Mad Max",
        "the town": "The Town",
        "abbot elementary": "Abbot Elementary",
        "abbott elementary": "Abbot Elementary",
    }

    def canonicalize_title(raw_title: str) -> str:
        cleaned = str(raw_title).strip()
        return title_normalization.get(cleaned.lower(), cleaned)

    stimulus_map = pd.read_csv(project_root / "data" / "stimulus_rename.csv")
    stimulus_map["group_letter"] = stimulus_map["group"].str.extract(r"([A-F])", expand=False)
    stimulus_map["title_clean"] = stimulus_map["title"].astype(str).str.strip()

    group_title_form_lookup = {}
    default_form_per_title = {}

    for row in stimulus_map.itertuples():
        if pd.isna(row.group_letter) or pd.isna(row.title_clean) or pd.isna(row.form):
            continue
        canonical_title = canonicalize_title(row.title_clean)
        form_value = str(row.form).title()
        group_title_form_lookup[(row.group_letter, canonical_title)] = form_value
        default_form_per_title.setdefault(canonical_title, form_value)

    screening_records = []

    for _, row in screening_raw.iterrows():
        respondent_id = row.get("respondent")
        if respondent_id is None:
            continue
        respondent_id = str(respondent_id).strip()
        group_letter = respondent_groups.get(respondent_id)
        for column in screening_value_columns:
            value = row.get(column)
            if pd.isna(value) or value == "":
                continue
            base_part, _, suffix_part = column.partition("_Survey_Familiarity_")
            if not suffix_part:
                continue
            question_code = suffix_part.strip()
            canonical_title = canonicalize_title(base_part.strip())
            form_value = None
            if group_letter:
                form_value = group_title_form_lookup.get((group_letter, canonical_title))
            if form_value is None:
                form_value = default_form_per_title.get(canonical_title, "Long")
            target_column = f"{form_value}_{canonical_title}_Screening_Familiarity_{question_code}"
            screening_records.append({
                "respondent": respondent_id,
                "target_column": target_column,
                "value": pd.to_numeric(value, errors="coerce")
            })

    if screening_records:
        screening_features = (
            pd.DataFrame(screening_records)
            .pivot_table(index="respondent", columns="target_column", values="value", aggfunc="first")
            .reset_index()
        )
        screening_features.columns.name = None
        survey_features = survey_features.merge(screening_features, on="respondent", how="left")

survey_features

,respondent,survey_group,survey_study,survey_gender,survey_age,survey_file,Long_The Town_Survey_Enjoyment_E1,Long_The Town_Survey_Enjoyment_E12,Long_The Town_Survey_Enjoyment_E14,Long_The Town_Survey_Enjoyment_E15,...,Long_The Town_Screening_Familiarity_F2,Short_Abbot Elementary_Screening_Familiarity_C1,Short_Abbot Elementary_Screening_Familiarity_F1,Short_Abbot Elementary_Screening_Familiarity_F2,Short_Mad Max_Screening_Familiarity_C1,Short_Mad Max_Screening_Familiarity_F1,Short_Mad Max_Screening_Familiarity_F2,Short_The Town_Screening_Familiarity_C1,Short_The Town_Screening_Familiarity_F1,Short_The Town_Screening_Familiarity_F2
0,83,Default,Group A,MALE,69,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,NaN,5.0,4.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,81,Default,Group A,FEMALE,24,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,2.0,1.0,1.0,NaN,4.0,3.0,1.0,NaN,NaN,NaN
2,99,Default,Group A,FEMALE,25,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,4.0,4.0,4.0,4.0,...,NaN,2.0,2.0,NaN,4.0,3.0,1.0,NaN,NaN,NaN
3,52,Default,Group A,FEMALE,50,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,4.0,5.0,5.0,...,NaN,1.0,1.0,NaN,2.0,2.0,NaN,NaN,NaN,NaN
4,8,Default,Group A,OTHER,51,MERGED_SURVEY_RESPONSE_MATRIX-A1.txt,5.0,5.0,5.0,5.0,...,1.0,4.0,3.0,1.0,4.0,3.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,85,Default,Group F,FEMALE,34,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,0.0,0.0,NaN
78,70,Default,Group F,FEMALE,61,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN
79,96,Default,Group F,FEMALE,29,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,4.0,3.0,1.0
80,41,Default,Group F,FEMALE,53,MERGED_SURVEY_RESPONSE_MATRIX-F2.txt,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,4.0,3.0,1.0


In [26]:
from pathlib import Path

def _safe_write_csv(df: pd.DataFrame, path: Path) -> Path:
    try:
        df.to_csv(path, index=False)
        return path
    except PermissionError:
        fallback = path.with_name(f"{path.stem}_{pd.Timestamp.utcnow().strftime('%Y%m%d%H%M%S')}.csv")
        df.to_csv(fallback, index=False)
        print(f"Permission denied for {path}. Saved to {fallback.name} instead.")
        return fallback

results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

stage1_path = results_dir / "uv_stage1.csv"
if not stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 output not found at {stage1_path}.")
uv_stage1_base = pd.read_csv(stage1_path)
uv_stage1_base["respondent"] = uv_stage1_base["respondent"].astype(str).str.strip()

survey_features = survey_features.copy()
survey_features["respondent"] = survey_features["respondent"].astype(str).str.strip()

uv_stage2 = uv_stage1_base.merge(survey_features, on="respondent", how="left")

issues_records: list[dict] = []
feature_ids = survey_features["respondent"].dropna().astype(str).str.strip()
expected_ids = uv_stage1_base["respondent"].astype(str).str.strip()

missing_ids = sorted(set(expected_ids) - set(feature_ids))
for respondent in missing_ids:
    issues_records.append({
        "respondent": respondent,
        "issue": "stage2_features_missing",
    })

duplicate_mask = feature_ids.duplicated(keep=False)
if duplicate_mask.any():
    duplicates = feature_ids[duplicate_mask]
    for respondent in sorted(duplicates.unique()):
        count = int((feature_ids == respondent).sum())
        issues_records.append({
            "respondent": respondent,
            "issue": f"stage2_duplicate_records_count_{count}",
        })

issues_df = pd.DataFrame(issues_records)

features_path = _safe_write_csv(survey_features, results_dir / "uv_stage2_features.csv")
open_ended_path = _safe_write_csv(survey_open_ended, results_dir / "uv_stage2_open_ended.csv")
uv_path = _safe_write_csv(uv_stage2, results_dir / "uv_stage2.csv")

issues_path = results_dir / "uv_stage2_issues.csv"
if issues_df.empty:
    if issues_path.exists():
        issues_path.unlink()
    print("No Stage 2 survey issues detected.")
else:
    _safe_write_csv(issues_df, issues_path)
    print(f"Logged {issues_df.shape[0]} Stage 2 issues to {issues_path.name}.")

print(
    f"Stage 2 survey features saved to {features_path.name} with {survey_features.shape[0]} respondents and "
    f"{survey_features.shape[1] - 1} feature columns."
)
print(f"Open-ended responses archived to {open_ended_path.name}.")
print(f"Unified view with Stage 2 survey data exported to {uv_path.name}.")

uv_stage2


Logged 1 Stage 2 issues to uv_stage2_issues.csv.
Stage 2 survey features saved to uv_stage2_features.csv with 82 respondents and 356 feature columns.
Open-ended responses archived to uv_stage2_open_ended.csv.
Unified view with Stage 2 survey data exported to uv_stage2.csv.


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,...,Long_The Town_Screening_Familiarity_F2,Short_Abbot Elementary_Screening_Familiarity_C1,Short_Abbot Elementary_Screening_Familiarity_F1,Short_Abbot Elementary_Screening_Familiarity_F2,Short_Mad Max_Screening_Familiarity_C1,Short_Mad Max_Screening_Familiarity_F1,Short_Mad Max_Screening_Familiarity_F2,Short_The Town_Screening_Familiarity_C1,Short_The Town_Screening_Familiarity_F1,Short_The Town_Screening_Familiarity_F2
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",...,1.0,0.0,0.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",...,NaN,4.0,3.0,1.0,4.0,3.0,1.0,NaN,NaN,NaN
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",...,NaN,1.0,1.0,NaN,4.0,3.0,1.0,NaN,NaN,NaN
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",...,1.0,8.0,4.0,4.0,8.0,4.0,4.0,NaN,NaN,NaN
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",...,NaN,1.0,1.0,NaN,5.0,3.0,2.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,005_50.csv,F,50,10/14/2025,09:54:03,63,60-69,Male,Black/African American,"$60,000 or more per year",...,NaN,NaN,NaN,NaN,6.0,4.0,2.0,6.0,3.0,3.0
79,004_60.csv,F,60,10/15/2025,09:34:06,66,60-69,Male,White,"$35,000  $60,000 per year",...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN
80,003_70.csv,F,70,10/16/2025,09:49:14,61,60-69,Female,Black/African American,"$35,000  $60,000 per year",...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN
81,002_85.csv,F,85,10/17/2025,14:37:41,34,28-43,Female,White,"$60,000 or more per year",...,NaN,NaN,NaN,NaN,4.0,3.0,1.0,0.0,0.0,NaN


In [27]:
print(survey_features.loc[(survey_features["respondent"].astype(int)==82)]['Long_Abbot Elementary_Survey_EnjoymentComposite_Sum'])
print(survey_features.loc[(survey_features["respondent"].astype(int)==82)]['Long_Abbot Elementary_Survey_Enjoyment_E18'])
print(survey_features.loc[(survey_features["respondent"].astype(int)==82)]['Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected'])

31    25.0
Name: Long_Abbot Elementary_Survey_EnjoymentComposite_Sum, dtype: float64
31    2.0
Name: Long_Abbot Elementary_Survey_Enjoyment_E18, dtype: float64
31    23.0
Name: Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected, dtype: float64


In [28]:
prefix = "Long_Abbot Elementary"
columns_to_show = [
    f"{prefix}_Survey_EnjoymentComposite_Sum",
    f"{prefix}_Survey_EnjoymentComposite_Corrected",
    f"{prefix}_Survey_EnjoymentComposite_Count",
    f"{prefix}_Survey_EnjoymentComposite_Normalized",
    f"{prefix}_Survey_EnjoymentComposite_NormalizedCorrected",
    f"{prefix}_Survey_EnjoymentComposite_Mean",
]
comparison = survey_features.loc[
    survey_features[f"{prefix}_Survey_EnjoymentComposite_Count"].gt(0),
    columns_to_show
].head(10).copy()
comparison["normalized_from_corrected"] = np.clip(
    (comparison[f"{prefix}_Survey_EnjoymentComposite_Corrected"]
     - comparison[f"{prefix}_Survey_EnjoymentComposite_Count"])
    / (4.0 * comparison[f"{prefix}_Survey_EnjoymentComposite_Count"]),
    0,
    1,
 )
comparison

,Long_Abbot Elementary_Survey_EnjoymentComposite_Sum,Long_Abbot Elementary_Survey_EnjoymentComposite_Corrected,Long_Abbot Elementary_Survey_EnjoymentComposite_Count,Long_Abbot Elementary_Survey_EnjoymentComposite_Normalized,Long_Abbot Elementary_Survey_EnjoymentComposite_NormalizedCorrected,Long_Abbot Elementary_Survey_EnjoymentComposite_Mean,normalized_from_corrected
22,41.0,41.0,12,0.604167,0.604167,3.416667,0.604167
23,51.0,53.0,12,0.812500,0.854167,4.416667,0.854167
24,50.0,54.0,12,0.791667,0.875000,4.500000,0.875000
25,45.0,49.0,12,0.687500,0.770833,4.083333,0.770833
26,51.0,55.0,12,0.812500,0.895833,4.583333,0.895833
27,35.0,37.0,12,0.479167,0.520833,3.083333,0.520833
28,19.0,15.0,12,0.145833,0.062500,1.250000,0.062500
29,19.0,15.0,12,0.145833,0.062500,1.250000,0.062500
30,55.0,55.0,12,0.895833,0.895833,4.583333,0.895833
31,25.0,23.0,12,0.270833,0.229167,1.916667,0.229167


## Stage 3: Post Questionnaire
The seven-day follow-up instrument captures delayed recall, comprehension, and broader post-viewing perceptions. This section documents how the Post questionnaire exports extend the unified view (`uv`) with respondent-level memory, confidence, and engagement measures.

### Planned Workflow
- Inventory every group-level Post questionnaire export under `data/Recall/`, ensuring respondent IDs stay string-typed for clean joins.
- Harmonize column names via `data/post_survey_map.csv` so variants or duplicate headers collapse to a single `question_code`.
- Engineer response features (e.g., accuracy, confidence, free-text summaries) while following the `{form}_{title}_Post_{metric}_{method}` naming pattern.
- Merge the Post feature frame onto the Stage 1 base (preserving respondent metadata) and log gaps to a dedicated issues table.
- Validate the enriched unified view and emit Stage 3 outputs (`uv_stage3.csv`, `uv_stage3_issues.csv`) to `results/` with timestamped fallbacks when needed.


In [29]:
from pathlib import Path
import pandas as pd
import re

post_map_path = project_root / "data" / "post_survey_map.csv"
post_map_df = pd.read_csv(post_map_path)

code_pattern = re.compile(r"^\s*([0-9]+(?:\.[0-9]+)*)")

def extract_question_code(text: str):
    if pd.isna(text):
        return None
    match = code_pattern.match(str(text))
    return match.group(1) if match else None

if "question_code" not in post_map_df.columns:
    post_map_df.insert(1, "question_code", post_map_df["question"].apply(extract_question_code))
else:
    post_map_df["question_code"] = post_map_df["question"].apply(extract_question_code)

post_map_df.to_csv(post_map_path, index=False)
missing_codes = post_map_df["question_code"].isna().sum()
print(
    f"Updated {post_map_path.name} with question_code column; "
    f"{missing_codes} rows are missing a parsed code."
)
post_map_df.head(10)

Updated post_survey_map.csv with question_code column; 7 rows are missing a parsed code.


,question,question_code,type,subscale,category,accuracy,Group A,Group B,Group C,Group D,Group E,Group F
0,1.1. Did you see this scene in the videos you ...,1.1,binary,recognition,key,hit,Mad Max,The Town,The Town,Abbot Elementary,Mad Max,Mad Max
1,1.2. How confident are you in your answer? Ple...,1.2,likert,recognition,key,hit,Mad Max,The Town,The Town,Abbot Elementary,Mad Max,Mad Max
2,1.3. This scene was part of the videos you wat...,1.3,multiple,recognition,key,hit,Mad Max,The Town,The Town,Abbot Elementary,Mad Max,Mad Max
3,2.1. Did you see this scene in the videos you ...,2.1,binary,recognition,fake,miss,The Dark Knight,The Dark Knight,Goodfellas,Goodfellas,Wonder Woman,Wonder Woman
4,2.2. How confident are you in your answer? Ple...,2.2,likert,recognition,fake,miss,The Dark Knight,The Dark Knight,Goodfellas,Goodfellas,Wonder Woman,Wonder Woman
5,2.3. This scene was not in the videos you watc...,2.3,multiple,recognition,fake,miss,The Dark Knight,The Dark Knight,Goodfellas,Goodfellas,Wonder Woman,Wonder Woman
6,3.1. Did you see this scene in the videos you ...,3.1,binary,recognition,distractor,hit,Titanic,The Notebook,The Notebook,Schitts Creek,Schitts Creek,Titanic
7,3.2. How confident are you in your answer? Ple...,3.2,likert,recognition,distractor,hit,Titanic,The Notebook,The Notebook,Schitts Creek,Schitts Creek,Titanic
8,3.3. This scene was part of the videos you wat...,3.3,likert,recognition,distractor,hit,Titanic,The Notebook,The Notebook,Schitts Creek,Schitts Creek,Titanic
9,4.1. Did you see this scene in the videos you ...,4.1,binary,recognition,key,hit,The Town,Mad Max,Abbot Elementary,The Town,Abbot Elementary,Abbot Elementary


In [30]:
import numpy as np
import re

post_data_dir = project_root / "data" / "Post"
if not post_data_dir.exists():
    raise FileNotFoundError(f"Post questionnaire directory not found at {post_data_dir}.")
post_files = sorted(
    path
    for path in post_data_dir.rglob("*.csv")
    if path.is_file() and not path.name.startswith("~$")
)
if not post_files:
    raise FileNotFoundError(f"No post questionnaire CSV files found under {post_data_dir}.")

def merge_duplicate_columns(df: pd.DataFrame) -> pd.DataFrame:
    def _sanitize(value):
        if pd.isna(value):
            return np.nan
        if isinstance(value, str):
            cleaned = value.strip()
            return cleaned if cleaned else np.nan
        return value
    merged = {}
    column_order = []
    for column in df.columns:
        normalized = re.sub(r"\s+", " ", str(column)).strip()
        series = df[column].astype("object").map(_sanitize)
        if normalized not in merged:
            merged[normalized] = series
            column_order.append(normalized)
        else:
            merged[normalized] = merged[normalized].combine_first(series)
    return pd.DataFrame({name: merged[name] for name in column_order})

post_map_full = pd.read_csv(project_root / "data" / "post_survey_map.csv")
if "question_code" not in post_map_full.columns:
    post_map_full.insert(1, "question_code", post_map_full["question"].apply(extract_question_code))
else:
    post_map_full["question_code"] = post_map_full["question"].apply(extract_question_code)
post_map_full["subscale"] = post_map_full["subscale"].astype(str)

CATEGORY_RENAME = {
    "key": "wb-key",
    "seen": "wb-notKeySeen",
    "unseen": "wb-notKeyUnseen",
    "fake": "distractor",
    "distractor": "comp-key",
    "distractor2": "comp-notKeySeen",
}

STAT_LABELS = {
    "count": "Count",
    "sum": "Sum",
    "mean": "Mean",
    "normalized_mean": "NormalizedMean",
}
RECOGNITION_BINARY_MAX = 2.0
RECOGNITION_CONFIDENCE_MAX = 4.0
RECOGNITION_COMPOSITE_MAX = RECOGNITION_BINARY_MAX * RECOGNITION_CONFIDENCE_MAX
FORM_CATEGORY_KEYS = {"key", "seen"}
NON_FORM_CATEGORY_KEYS = {"unseen", "fake", "distractor", "distractor2"}

uv_stage1_path = project_root / "results" / "uv_stage1.csv"
if not uv_stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 dataset not found at {uv_stage1_path}.")
uv_stage1 = pd.read_csv(uv_stage1_path)
uv_stage1["respondent"] = uv_stage1["respondent"].astype(str).str.strip()
uv_stage1["group"] = uv_stage1["group"].astype(str).str.strip().str.upper()

uv_stage2_path = project_root / "results" / "uv_stage2.csv"
uv_stage2_cached = None
if uv_stage2_path.exists():
    uv_stage2_cached = pd.read_csv(uv_stage2_path)
    uv_stage2_cached["respondent"] = uv_stage2_cached["respondent"].astype(str).str.strip()

uv_stage1_lookup_df = (
    uv_stage1
    .loc[:, ["respondent", "group", "Short Form", "Long Form"]]
    .dropna(subset=["respondent"])
    .assign(respondent=lambda df: df["respondent"].astype(str).str.strip())
    .drop_duplicates("respondent", keep="last")
)
uv_stage1_lookup = uv_stage1_lookup_df.set_index("respondent").to_dict("index")

recognition_map = post_map_full.loc[
    post_map_full["subscale"].str.lower() == "recognition",
    :,
]
recognition_map = recognition_map.loc[recognition_map["question_code"].notna()].copy()
recognition_map["question_code"] = recognition_map["question_code"].astype(str).str.strip()
recognition_map = recognition_map.loc[recognition_map["question_code"].str.match(r"^\d+\.[12]$")]

valid_question_codes = set(recognition_map["question_code"].unique())

group_columns = [col for col in recognition_map.columns if col.startswith("Group ")]

TITLE_NORMALIZATION = {
    "abbott elementary": "Abbot Elementary",
    "abbot elementary": "Abbot Elementary",
    "schitts creek": "Schittss Creek",
    "schittss creek": "Schittss Creek",
    "mad max fury road": "Mad Max",
    "mad max": "Mad Max",
}

def canonicalize_title(raw_title: str) -> str:
    if pd.isna(raw_title):
        return ""
    cleaned = str(raw_title).strip()
    if not cleaned:
        return ""
    lookup_key = cleaned.lower()
    return TITLE_NORMALIZATION.get(lookup_key, cleaned)

respondent_exposures: dict[str, dict[str, set[str]]] = {}
for respondent_id, info in uv_stage1_lookup.items():
    exposures: dict[str, set[str]] = {}
    long_title = canonicalize_title(info.get("Long Form", ""))
    if long_title:
        exposures.setdefault("Long", set()).add(long_title)
    short_title = canonicalize_title(info.get("Short Form", ""))
    if short_title:
        exposures.setdefault("Short", set()).add(short_title)
    if exposures:
        respondent_exposures[str(respondent_id)] = exposures

meta_lookup = {}
for _, row in recognition_map.iterrows():
    q_code = row["question_code"]
    q_root, q_suffix = q_code.split(".")
    category = str(row.get("category", "")).strip()
    accuracy = str(row.get("accuracy", "")).strip().lower()
    for group_col in group_columns:
        group_letter = group_col.replace("Group ", "").strip().upper()
        title_value = canonicalize_title(row.get(group_col, ""))
        meta_lookup[(group_letter, q_root, q_suffix)] = {
            "title": title_value,
            "category": category,
            "accuracy": accuracy,
        }

stimulus_map = pd.read_csv(project_root / "data" / "stimulus_rename.csv")
stimulus_map["group_letter"] = stimulus_map["group"].str.extract(r"([A-F])", expand=False)
stimulus_map["title_clean"] = stimulus_map["title"].map(canonicalize_title)
stimulus_map["form_clean"] = stimulus_map["form"].astype(str).str.title()

group_title_form_lookup = {
    (row.group_letter, row.title_clean): row.form_clean
    for row in stimulus_map.itertuples()
    if isinstance(row.group_letter, str) and isinstance(row.title_clean, str) and row.title_clean
}

if uv_stage2_cached is not None:
    uv_columns = pd.Index(uv_stage2_cached.columns)
else:
    uv_columns = pd.Index(uv_stage1.columns)

yes_values = {"yes", "y", "true", "1"}
no_values = {"no", "n", "false", "0"}

confidence_pattern = re.compile(r"^\s*([0-9]+(?:\.[0-9]+)?)")

def extract_scalar(value):
    if isinstance(value, pd.Series):
        non_null = value.dropna()
        if non_null.empty:
            return np.nan
        return non_null.iloc[0]
    return value

def parse_yes_no(value) -> float:
    value = extract_scalar(value)
    if pd.isna(value):
        return np.nan
    text = str(value).strip().lower()
    if not text:
        return np.nan
    if text in yes_values:
        return 1.0
    if text in no_values:
        return 0.0
    return np.nan

def parse_confidence(value) -> float:
    value = extract_scalar(value)
    if pd.isna(value):
        return np.nan
    text = str(value).strip()
    if not text:
        return np.nan
    match = confidence_pattern.match(text)
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return np.nan
    try:
        return float(text)
    except ValueError:
        return np.nan

def resolve_form(respondent_id: str, group_letter: str, title: str) -> tuple[str, list[str]]:
    notes: list[str] = []
    exposures = uv_stage1_lookup.get(respondent_id, {})
    long_title = canonicalize_title(exposures.get("Long Form", "")) if exposures else ""
    short_title = canonicalize_title(exposures.get("Short Form", "")) if exposures else ""
    canonical = canonicalize_title(title)
    if canonical and canonical == long_title:
        return "Long", notes
    if canonical and canonical == short_title:
        return "Short", notes
    group_clean = (group_letter or "").strip().upper()
    if canonical:
        mapped = group_title_form_lookup.get((group_clean, canonical))
        if mapped:
            notes.append("form_from_group_stimulus_map")
            return mapped, notes
        has_short = any(col.startswith(f"Short_{canonical}") for col in uv_columns)
        has_long = any(col.startswith(f"Long_{canonical}") for col in uv_columns)
        if has_short and not has_long:
            notes.append("form_inferred_short_from_uv")
            return "Short", notes
        if has_long and not has_short:
            notes.append("form_inferred_long_from_uv")
            return "Long", notes
        if has_short and has_long:
            notes.append("form_ambiguous_default_short")
            return "Short", notes
    notes.append("form_unresolved_default_short")
    return "Short", notes

recognition_records: list[dict] = []
issue_records: list[dict] = []
respondent_post_paths: dict[str, str] = {}
excluded_group_mismatch = set()

for csv_path in post_files:
    file_group_match = re.search(r"Group\s+([A-F])", csv_path.stem, re.IGNORECASE)
    fallback_group_letter = file_group_match.group(1).upper() if file_group_match else ""
    df_raw = pd.read_csv(csv_path, dtype=str)
    df_merged = merge_duplicate_columns(df_raw)
    rename_map = {}
    for column in df_merged.columns:
        q_code = extract_question_code(column)
        if q_code and q_code in valid_question_codes:
            rename_map[column] = q_code
    df_standard = df_merged.rename(columns=rename_map)
    respondent_col = next(((
        col for col in df_standard.columns
        if "participant number" in col.lower()
    )), None)
    if respondent_col is None:
        raise KeyError(f"Participant identifier column not found in {csv_path.name}")
    df_standard["respondent"] = df_standard[respondent_col].astype(str).str.strip()
    df_standard["respondent"] = df_standard["respondent"].replace({"": np.nan, "nan": np.nan})
    df_standard["respondent"] = df_standard["respondent"].str.replace(r"\.0$", "", regex=True)
    if "Timestamp" in df_standard.columns:
        df_standard["timestamp_iso"] = pd.to_datetime(df_standard["Timestamp"], errors="coerce")
    else:
        df_standard["timestamp_iso"] = pd.NaT

    for _, row in df_standard.iterrows():
        respondent_id = row.get("respondent")
        if pd.isna(respondent_id):
            continue
        respondent_id = str(respondent_id).strip()
        if not respondent_id:
            continue
        respondent_info = uv_stage1_lookup.get(respondent_id, {})
        stage1_group = str(respondent_info.get("group", "")).strip().upper()
        post_group_letter = fallback_group_letter
        if stage1_group and post_group_letter and stage1_group != post_group_letter:
            if respondent_id not in excluded_group_mismatch:
                issue_records.append({
                    "respondent": respondent_id,
                    "group": stage1_group,
                    "question_number": np.nan,
                    "title": "",
                    "issue": f"post_group_mismatch_uv_{stage1_group}_file_{post_group_letter}",
                    "source_file": csv_path.name,
                })
            excluded_group_mismatch.add(respondent_id)
            continue
        group_letter = stage1_group or post_group_letter
        if not group_letter:
            issue_records.append({
                "respondent": respondent_id,
                "group": "",
                "question_number": np.nan,
                "title": "",
                "issue": "group_missing_in_stage1_and_filename",
                "source_file": csv_path.name,
            })
            continue
        if respondent_id not in respondent_post_paths:
            try:
                relative_path = csv_path.relative_to(project_root)
                respondent_post_paths[respondent_id] = relative_path.as_posix()
            except ValueError:
                respondent_post_paths[respondent_id] = csv_path.as_posix()
        for q_num in map(str, range(1, 13)):
            base_meta = meta_lookup.get((group_letter, q_num, "1"))
            conf_meta = meta_lookup.get((group_letter, q_num, "2"))
            if base_meta is None or conf_meta is None:
                continue
            binary_value = row.get(f"{q_num}.1")
            confidence_value = row.get(f"{q_num}.2")
            yes_no = parse_yes_no(binary_value)
            confidence = parse_confidence(confidence_value)
            issues_here: list[str] = []
            if np.isnan(yes_no):
                issues_here.append("binary_response_missing_or_unrecognized")
            accuracy = base_meta.get("accuracy", "")
            expected_yes = accuracy == "hit"
            base_score_raw = np.nan
            base_score = np.nan
            if not np.isnan(yes_no):
                answered_yes = bool(yes_no)
                base_score_raw = RECOGNITION_BINARY_MAX if answered_yes == expected_yes else 0.0
                base_score = base_score_raw / RECOGNITION_BINARY_MAX
            if np.isnan(confidence):
                issues_here.append("confidence_missing_or_unrecognized")
            composite_raw = np.nan
            composite = np.nan
            if not np.isnan(base_score_raw) and not np.isnan(confidence):
                composite_raw = base_score_raw * confidence
                composite = composite_raw / RECOGNITION_COMPOSITE_MAX
            form_value, form_notes = resolve_form(respondent_id, group_letter, base_meta.get("title", ""))
            issues_here.extend(form_notes)
            question_int = int(float(q_num))
            title_segment = base_meta.get("title", "").strip() or f"Question {question_int}"
            composite_name = f"{form_value}_{title_segment}_Post_RecognitionComposite_Q{question_int:02d}"
            base_name = f"{form_value}_{title_segment}_Post_Recognition_Q{question_int}-1"
            confidence_name = f"{form_value}_{title_segment}_Post_Recognition_Q{question_int}-2"
            records_to_add = [
                ("composite", composite_name, composite),
                ("binary", base_name, base_score),
                ("confidence", confidence_name, confidence),
            ]
            for metric_label, feature_name, feature_value in records_to_add:
                recognition_records.append({
                    "respondent": respondent_id,
                    "group": group_letter,
                    "question_number": question_int,
                    "title": base_meta.get("title", ""),
                    "category": base_meta.get("category", ""),
                    "accuracy": accuracy,
                    "form": form_value,
                    "metric": metric_label,
                    "column_name": feature_name,
                    "value": feature_value,
                    "timestamp": row.get("timestamp_iso", pd.NaT),
                })
            for issue in issues_here:
                issue_records.append({
                    "respondent": respondent_id,
                    "group": group_letter,
                    "question_number": question_int,
                    "title": base_meta.get("title", ""),
                    "issue": issue,
                    "source_file": csv_path.name,
                })

recognition_df = pd.DataFrame(recognition_records)
if recognition_df.empty:
    raise ValueError("No recognition responses were parsed from post questionnaire files.")
recognition_df = recognition_df.sort_values(["respondent", "column_name", "timestamp"])
recognition_df = recognition_df.drop_duplicates(["respondent", "column_name"], keep="last")

recognition_features = (
    recognition_df
    .pivot(index="respondent", columns="column_name", values="value")
    .sort_index(axis=1)
    .reset_index()
)
recognition_features.columns.name = None

composite_subset = recognition_df.loc[recognition_df["metric"] == "composite"].copy()
if not composite_subset.empty:
    composite_subset["category_lower"] = (
        composite_subset["category"]
        .astype(str)
        .str.strip()
        .str.lower()
        .replace("", np.nan)
    )
    composite_subset = composite_subset.loc[composite_subset["category_lower"].notna()]
    composite_subset["title_clean"] = composite_subset["title"].map(canonicalize_title)
    composite_subset["respondent"] = composite_subset["respondent"].astype(str).str.strip()

    form_subset = composite_subset.loc[composite_subset["category_lower"].isin(FORM_CATEGORY_KEYS)].copy()
    if not form_subset.empty:
        def _match_form(row) -> str | None:
            exposures = respondent_exposures.get(row["respondent"], {})
            for form_label, titles in exposures.items():
                if row["title_clean"] in titles:
                    return form_label
            return None

        form_subset["aligned_form"] = form_subset.apply(_match_form, axis=1)
        unmatched_mask = form_subset["aligned_form"].isna()
        if unmatched_mask.any():
            for row in form_subset.loc[unmatched_mask, ["respondent", "question_number", "title", "category_lower"]].itertuples(index=False):
                respondent_key = str(row.respondent)
                issue_records.append({
                    "respondent": respondent_key,
                    "group": uv_stage1_lookup.get(respondent_key, {}).get("group", ""),
                    "question_number": int(row.question_number) if not pd.isna(row.question_number) else np.nan,
                    "title": row.title,
                    "issue": "post_form_title_not_in_stage1_exposures",
                    "source_file": respondent_post_paths.get(respondent_key, ""),
                })
            form_subset = form_subset.loc[~unmatched_mask]
        if not form_subset.empty:
            form_agg = (
                form_subset
                .groupby(["respondent", "aligned_form", "category_lower"])["value"]
                .agg(count="count", sum="sum", mean="mean")
                .reset_index()
            )
            if not form_agg.empty:
                form_agg["normalized_mean"] = form_agg["mean"].astype(float).clip(0, 1)
                form_long = form_agg.melt(
                    id_vars=["respondent", "aligned_form", "category_lower"],
                    value_vars=["count", "sum", "mean", "normalized_mean"],
                    var_name="statistic",
                    value_name="stat_value",
                )
                form_long["column_name"] = form_long.apply(
                    lambda r: f"{r['aligned_form']}_{r['category_lower']}_Post_Recognition_{STAT_LABELS[r['statistic']]}"
                    , axis=1,
                )
                form_wide = form_long.pivot(index="respondent", columns="column_name", values="stat_value").reset_index()
                form_wide.columns.name = None
                recognition_features = recognition_features.merge(form_wide, on="respondent", how="left")

    non_form_subset = composite_subset.loc[composite_subset["category_lower"].isin(NON_FORM_CATEGORY_KEYS)].copy()
    if not non_form_subset.empty:
        non_form_agg = (
            non_form_subset
            .groupby(["respondent", "category_lower"])["value"]
            .agg(count="count", sum="sum", mean="mean")
            .reset_index()
        )
        if not non_form_agg.empty:
            non_form_agg["normalized_mean"] = non_form_agg["mean"].astype(float).clip(0, 1)
            non_form_long = non_form_agg.melt(
                id_vars=["respondent", "category_lower"],
                value_vars=["count", "sum", "mean", "normalized_mean"],
                var_name="statistic",
                value_name="stat_value",
            )
            non_form_long["column_name"] = non_form_long.apply(
                lambda r: f"{r['category_lower']}_Post_Recognition_{STAT_LABELS[r['statistic']]}"
                , axis=1,
            )
            non_form_wide = non_form_long.pivot(index="respondent", columns="column_name", values="stat_value").reset_index()
            non_form_wide.columns.name = None
            recognition_features = recognition_features.merge(non_form_wide, on="respondent", how="left")

if respondent_post_paths:
    post_path_df = (
        pd.Series(respondent_post_paths, name="post_survey_source_path")
        .to_frame()
        .reset_index()
        .rename(columns={"index": "respondent"})
    )
    recognition_features = recognition_features.merge(post_path_df, on="respondent", how="left")

feature_columns = [col for col in recognition_features.columns if col != "respondent"]
composite_columns = [col for col in feature_columns if "_Post_RecognitionComposite_" in col]
raw_columns = [
    col
    for col in feature_columns
    if "_Post_Recognition_" in col and "Composite" not in col and not col.endswith("source_path")
]

issues_df = pd.DataFrame(issue_records)
if not issues_df.empty:
    issues_df = issues_df.sort_values(["respondent", "question_number", "issue"])

if excluded_group_mismatch:
    def _mismatch_sort_key(value: str):
        text = str(value)
        if text.isdigit():
            return (0, int(text))
        return (1, text)
    excluded_display = ", ".join(sorted(excluded_group_mismatch, key=_mismatch_sort_key))
    print(
        f"Skipped {len(excluded_group_mismatch)} respondent(s) due to post group mismatch: {excluded_display}."
    )

print(
    f"Parsed {recognition_df.shape[0]} recognition feature rows across "
    f"{len(composite_columns)} composite columns and {len(raw_columns)} raw response columns "
    f"for {recognition_features.shape[0]} respondents.",
)
sample_columns = composite_columns[:3] + raw_columns[:3]
if sample_columns:
    print("Sample feature columns:")
    for name in sample_columns:
        print(f"  {name}")
recognition_features.head()


Skipped 3 respondent(s) due to post group mismatch: 6, 116, 117.
Parsed 2880 recognition feature rows across 39 composite columns and 110 raw response columns for 80 respondents.
Sample feature columns:
  Long_Abbot Elementary_Post_RecognitionComposite_Q04
  Long_Abbot Elementary_Post_RecognitionComposite_Q07
  Long_Abbot Elementary_Post_RecognitionComposite_Q09
  Long_Abbot Elementary_Post_Recognition_Q4-1
  Long_Abbot Elementary_Post_Recognition_Q4-2
  Long_Abbot Elementary_Post_Recognition_Q7-1


,respondent,Long_Abbot Elementary_Post_RecognitionComposite_Q04,Long_Abbot Elementary_Post_RecognitionComposite_Q07,Long_Abbot Elementary_Post_RecognitionComposite_Q09,Long_Abbot Elementary_Post_Recognition_Q4-1,Long_Abbot Elementary_Post_Recognition_Q4-2,Long_Abbot Elementary_Post_Recognition_Q7-1,Long_Abbot Elementary_Post_Recognition_Q7-2,Long_Abbot Elementary_Post_Recognition_Q9-1,Long_Abbot Elementary_Post_Recognition_Q9-2,...,distractor_Post_Recognition_Sum,fake_Post_Recognition_Count,fake_Post_Recognition_Mean,fake_Post_Recognition_NormalizedMean,fake_Post_Recognition_Sum,unseen_Post_Recognition_Count,unseen_Post_Recognition_Mean,unseen_Post_Recognition_NormalizedMean,unseen_Post_Recognition_Sum,post_survey_source_path
0,1,1.0,0.75,NaN,1.0,4.0,1.0,3.0,NaN,NaN,...,0.00,2.0,0.3750,0.3750,0.75,1.0,0.00,0.00,0.00,data/Post/Group C_Post Viewing Questionnaire P...
1,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.75,4.0,0.8125,0.8125,3.25,1.0,0.75,0.75,0.75,data/Post/Group B_ Post Viewing Questionnaire ...
2,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.00,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group E_ Post Viewing Questionnaire ...
3,101,1.0,1.00,1.0,1.0,4.0,1.0,4.0,1.0,4.0,...,1.00,4.0,0.6250,0.6250,2.50,1.0,0.50,0.50,0.50,data/Post/Group C_ Post Viewing Questionnaire ...
4,102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group D_ Post Viewing Questionnaire ...


In [31]:
feature_columns = [col for col in recognition_features.columns if col != "respondent"]
composite_columns = [col for col in feature_columns if "_Post_RecognitionComposite_" in col]
raw_columns = [col for col in feature_columns if "_Post_Recognition_" in col and "Composite" not in col]

print(f"Total recognition feature columns: {len(feature_columns)}")
print(f"Composite columns ({len(composite_columns)} total):")
for name in composite_columns[:20]:
    print(f"  {name}")
if len(composite_columns) > 20:
    print("  ...")
print(f"Raw recognition columns ({len(raw_columns)} total):")
for name in raw_columns[:20]:
    print(f"  {name}")
if len(raw_columns) > 20:
    print("  ...")

Total recognition feature columns: 150
Composite columns (39 total):
  Long_Abbot Elementary_Post_RecognitionComposite_Q04
  Long_Abbot Elementary_Post_RecognitionComposite_Q07
  Long_Abbot Elementary_Post_RecognitionComposite_Q09
  Long_Mad Max_Post_RecognitionComposite_Q01
  Long_Mad Max_Post_RecognitionComposite_Q04
  Long_Mad Max_Post_RecognitionComposite_Q07
  Long_The Town_Post_RecognitionComposite_Q04
  Long_The Town_Post_RecognitionComposite_Q07
  Long_The Town_Post_RecognitionComposite_Q09
  Short_A Star Is Born_Post_RecognitionComposite_Q10
  Short_A Star Is Born_Post_RecognitionComposite_Q11
  Short_A Star Is Born_Post_RecognitionComposite_Q12
  Short_Abbot Elementary_Post_RecognitionComposite_Q01
  Short_Abbot Elementary_Post_RecognitionComposite_Q04
  Short_Abbot Elementary_Post_RecognitionComposite_Q05
  Short_Abbot Elementary_Post_RecognitionComposite_Q09
  Short_Friends_Post_RecognitionComposite_Q06
  Short_Friends_Post_RecognitionComposite_Q08
  Short_Goodfellas_Post_R

In [32]:
stage1_path = project_root / "results" / "uv_stage1.csv"
if not stage1_path.exists():
    raise FileNotFoundError(f"Stage 1 output not found at {stage1_path}.")
uv_stage1_base = pd.read_csv(stage1_path)
uv_stage1_base["respondent"] = uv_stage1_base["respondent"].astype(str).str.strip()

form_columns = [col for col in ("Short Form", "Long Form") if col in uv_stage1_base.columns]
forms_lookup = uv_stage1_base[["respondent", *form_columns]].copy() if form_columns else None
if forms_lookup is not None:
    forms_lookup["respondent"] = forms_lookup["respondent"].astype(str).str.strip()
    if forms_lookup["respondent"].duplicated().any():
        dup_count = forms_lookup["respondent"].duplicated(keep=False).sum()
        print(f"Warning: {dup_count} duplicate form records detected; using the last occurrence per respondent.")
        forms_lookup = forms_lookup.drop_duplicates(subset="respondent", keep="last")

recognition_features = recognition_features.copy()
recognition_features["respondent"] = recognition_features["respondent"].astype(str).str.strip()

uv_stage3 = uv_stage1_base.copy()
if forms_lookup is not None:
    uv_stage3 = uv_stage3.drop(columns=form_columns, errors="ignore")
    uv_stage3 = uv_stage3.merge(forms_lookup, on="respondent", how="left", validate="one_to_one")

uv_stage3 = uv_stage3.merge(recognition_features, on="respondent", how="left")
uv_stage3 = uv_stage3.sort_values("respondent").reset_index(drop=True)

results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)
stage3_path = results_dir / "uv_stage3.csv"
uv_stage3.to_csv(stage3_path, index=False)
print(f"Stage 3 UV shape: {uv_stage3.shape}; saved to {stage3_path.name}.")

issues_path = results_dir / "uv_stage3_issues.csv"
if issues_df.empty:
    if issues_path.exists():
        issues_path.unlink()
    print("No Stage 3 recognition parsing issues detected.")
else:
    issues_df.to_csv(issues_path, index=False)
    print(f"Logged {issues_df.shape[0]} recognition parsing issues to {issues_path.name}.")

uv_stage3


Stage 3 UV shape: (83, 167); saved to uv_stage3.csv.
Logged 599 recognition parsing issues to uv_stage3_issues.csv.


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,...,distractor_Post_Recognition_Sum,fake_Post_Recognition_Count,fake_Post_Recognition_Mean,fake_Post_Recognition_NormalizedMean,fake_Post_Recognition_Sum,unseen_Post_Recognition_Count,unseen_Post_Recognition_Mean,unseen_Post_Recognition_NormalizedMean,unseen_Post_Recognition_Sum,post_survey_source_path
0,008_1.csv,C,1,10/07/2025,10:32:16,63,60-69,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",...,0.00,2.0,0.3750,0.3750,0.75,1.0,0.00,0.00,0.00,data/Post/Group C_Post Viewing Questionnaire P...
1,005_10.csv,B,10,10/10/2025,13:17:14,65,60-69,Male,White,"$35,000  $60,000 per year",...,1.75,4.0,0.8125,0.8125,3.25,1.0,0.75,0.75,0.75,data/Post/Group B_ Post Viewing Questionnaire ...
2,005_100.csv,E,100,10/15/2025,19:14:06,25,18-27,Female,White,"$35,000  $60,000 per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group E_ Post Viewing Questionnaire ...
3,003_101.csv,C,101,10/16/2025,14:58:27,67,60-69,Male,Black/African American,"$60,000 or more per year",...,1.00,4.0,0.6250,0.6250,2.50,1.0,0.50,0.50,0.50,data/Post/Group C_ Post Viewing Questionnaire ...
4,001_102.csv,D,102,10/16/2025,16:35:40,37,28-43,Male,Black/African American,"$35,000  $60,000 per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group D_ Post Viewing Questionnaire ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,004_94.csv,D,94,10/15/2025,11:33:19,32,28-43,Male,White,"$35,000  $60,000 per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group D_ Post Viewing Questionnaire ...
79,004_96.csv,F,96,10/15/2025,14:32:00,29,28-43,Female,White,"$60,000 or more per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group F_ Post Viewing Questionnaire ...
80,004_97.csv,E,97,10/15/2025,17:41:01,32,28-43,Female,White,"$60,000 or more per year",...,2.00,4.0,1.0000,1.0000,4.00,1.0,0.00,0.00,0.00,data/Post/Group E_ Post Viewing Questionnaire ...
81,004_98.csv,A,98,10/15/2025,17:41:49,32,28-43,Male,White,"$35,000  $60,000 per year",...,2.00,4.0,0.8750,0.8750,3.50,1.0,0.50,0.50,0.50,data/Post/Group A_ Post Viewing Questionnaire ...


## UV Merge
This consolidation step reloads the Stage 2 and Stage 3 outputs, checks their respondent-level metadata against the Stage 1 base, and publishes a merged UV file plus an issues log highlighting any discrepancies.


In [33]:
from pathlib import Path
import pandas as pd

results_dir = project_root / "results"
stage1_path = results_dir / "uv_stage1.csv"
stage2_path = results_dir / "uv_stage2.csv"
stage3_path = results_dir / "uv_stage3.csv"

for required_path in [stage1_path, stage2_path, stage3_path]:
    if not required_path.exists():
        raise FileNotFoundError(f"Required file not found: {required_path}")

uv_stage1_base = pd.read_csv(stage1_path)
uv_stage2 = pd.read_csv(stage2_path)
uv_stage3 = pd.read_csv(stage3_path)

for df in (uv_stage1_base, uv_stage2, uv_stage3):
    df["respondent"] = df["respondent"].astype(str).str.strip()

baseline_cols = list(uv_stage1_base.columns)
baseline_set = set(baseline_cols)

issues_records: list[dict] = []

def log_issue(respondent: str, issue: str) -> None:
    issues_records.append({"respondent": respondent, "issue": issue})

for dataset_name, df in (("stage2", uv_stage2), ("stage3", uv_stage3)):
    duplicated = df[df["respondent"].duplicated(keep=False)]["respondent"].unique()
    for respondent in sorted(duplicated):
        log_issue(respondent, f"duplicate_in_{dataset_name}")

ids_stage1 = set(uv_stage1_base["respondent"])
ids_stage2 = set(uv_stage2["respondent"])
ids_stage3 = set(uv_stage3["respondent"])

for respondent in sorted(ids_stage1 - ids_stage2):
    log_issue(respondent, "missing_in_stage2")
for respondent in sorted(ids_stage1 - ids_stage3):
    log_issue(respondent, "missing_in_stage3")

shared_ids = sorted(ids_stage2 & ids_stage3)
stage2_baseline = uv_stage2.set_index("respondent")
stage3_baseline = uv_stage3.set_index("respondent")
baseline_columns = sorted((baseline_set - {"respondent"}) & set(stage2_baseline.columns) & set(stage3_baseline.columns))

for respondent in shared_ids:
    if respondent not in stage2_baseline.index or respondent not in stage3_baseline.index:
        continue
    if not baseline_columns:
        break
    row_stage2 = stage2_baseline.loc[respondent, baseline_columns]
    row_stage3 = stage3_baseline.loc[respondent, baseline_columns]
    for column in baseline_columns:
        value_stage2 = row_stage2[column]
        value_stage3 = row_stage3[column]
        if pd.isna(value_stage2) and pd.isna(value_stage3):
            continue
        if pd.isna(value_stage2) != pd.isna(value_stage3) or str(value_stage2) != str(value_stage3):
            log_issue(respondent, f"baseline_mismatch_{column}")

stage2_feature_cols = [col for col in uv_stage2.columns if col not in baseline_set]
stage3_feature_cols = [col for col in uv_stage3.columns if col not in baseline_set]

uv_merged = uv_stage1_base.copy()
uv_merged = uv_merged.merge(
    uv_stage2[["respondent", *stage2_feature_cols]],
    on="respondent", how="left"
 )
uv_merged = uv_merged.merge(
    uv_stage3[["respondent", *stage3_feature_cols]],
    on="respondent", how="left"
 )

merged_path = results_dir / "uv_merged.csv"
uv_merged.to_csv(merged_path, index=False)

issues_df = (
    pd.DataFrame(issues_records).sort_values(["respondent", "issue"])
    if issues_records
    else pd.DataFrame(columns=["respondent", "issue"])
 )
issues_path = results_dir / "merge_issues.csv"
if issues_df.empty:
    if issues_path.exists():
        issues_path.unlink()
    print("UV merge completed with no issues detected.")
else:
    issues_df.to_csv(issues_path, index=False)
    print(f"UV merge completed; logged {issues_df.shape[0]} issues to {issues_path.name}.")

uv_merged.head()

UV merge completed with no issues detected.


,source_file,group,respondent,date_study,time_study,age,age_group,gender,ethnicity,income_group,...,distractor_Post_Recognition_Sum,fake_Post_Recognition_Count,fake_Post_Recognition_Mean,fake_Post_Recognition_NormalizedMean,fake_Post_Recognition_Sum,unseen_Post_Recognition_Count,unseen_Post_Recognition_Mean,unseen_Post_Recognition_NormalizedMean,unseen_Post_Recognition_Sum,post_survey_source_path
0,003_104.csv,A,104,10/16/2025,18:09:03,59,44-59,Male,White,"$60,000 or more per year",...,1.0,4.0,0.6875,0.6875,2.75,1.0,0.00,0.00,0.00,data/Post/Group A_ Post Viewing Questionnaire ...
1,002_106.csv,A,106,10/16/2025,19:35:05,30,28-43,Male,White,"$60,000 or more per year",...,2.0,4.0,1.0000,1.0000,4.00,1.0,0.75,0.75,0.75,data/Post/Group A_ Post Viewing Questionnaire ...
2,001_116.csv,A,116,10/18/2025,12:37:40,19,18-27,Male,White,"$35,000  $60,000 per year",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,006_14.csv,A,14,10/11/2025,09:32:42,33,28-43,Male,Hispanic/Latino/Latina/Latinx,"$60,000 or more per year",...,1.0,4.0,0.8125,0.8125,3.25,1.0,1.00,1.00,1.00,data/Post/Group A_ Post Viewing Questionnaire ...
4,007_3.csv,A,3,10/10/2025,09:19:22,34,28-43,Female,White,"$60,000 or more per year",...,2.0,4.0,1.0000,1.0000,4.00,1.0,1.00,1.00,1.00,data/Post/Group A_ Post Viewing Questionnaire ...
